In [7]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import datetime
import os
import glob 

In [2]:
start_time = datetime.now()

In [4]:
dir = 'Y:\\delete\\US Environmental Protection Agency'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [5]:
ser = Service("C:/Users/AIMPAI0/Downloads/chromedriver.exe")
op = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "Y:\\delete\\US Environmental Protection Agency"}
op.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()  
driver.get("https://www.epa.gov/fera/dose-response-assessment-assessing-health-risks-associated-exposure-hazardous-air-pollutants")
driver.implicitly_wait(5)
#driver.find_element(by=By.XPATH, value='//*[@id="main"]/div/div[1]/article/div[2]/div/ul[1]/li[2]/span/a').click()
time.sleep(10)
driver.find_element(by=By.XPATH, value='//*[@id="main"]/div/div[1]/article/div[2]/div/ul[2]/li[2]/span/a').click()
time.sleep(10)
driver.close()

In [16]:
path = 'Y:\\delete\\US Environmental Protection Agency'
filenames = glob.glob(path + "/*.xlsx")
for f in filenames:
    dataset = pd.read_excel(f,skiprows=4)
#dataset = dataset[['CAS No','Specific information requirement\n']]
dataset = dataset.iloc[:, [1, 6]]
dataset.columns.values[0] = "CAS Number"
dataset.columns.values[1] = "status"

,CAS Number,status
0,75-07-0,B2
1,60-35-5,0
2,75-05-8,InI
3,98-86-2,D
4,107-02-8,InI
...,...,...
222,108-05-4,0
223,593-60-2,B2
224,75-01-4,CH
225,75-35-4,SE


In [17]:
CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Not in list'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)

result = pd.merge(dataset, CAS, how="outer", on="CAS Number")
result.status.fillna(result.value, inplace=True)
del result['value']

,CAS Number,status
0,75-07-0,B2
1,60-35-5,0
2,75-05-8,InI
3,98-86-2,D
4,107-02-8,InI
...,...,...
227,14807-96-6,Not in list
228,7631-86-91,Not in list
229,9002-84-0,Not in list
230,65-85-0,Not in list


In [25]:
result['Jurisdiction'] = 'US'
result['RegulatoryAgency'] = 'Environment Protection Aganecy (EPA)'
result['TypeOfRegulation'] = 'Health'
result['Criteria'] = 'Carcinegenity'

result['Description'] = 'Inadequate information or not evaluated'
result.loc[result['status'].isin(['A','CH']), 'Description'] = 'Carcinogenic to humans'
result.loc[result['status'].isin(['LH','B1','B2']), 'Description'] = 'Likely to be carcinogenic'
result.loc[result['status'].isin(['SE','C']), 'Description'] = 'Possible human carcinogen'
result.loc[result['status'].isin(['NH','E']), 'Description'] = 'Not likely to be carcinogen'


result['HazardClassification'] = 'Very Low'
result.loc[result['status'].isin(['A','CH']), 'HazardClassification'] = 'Very High'
result.loc[result['status'].isin(['LH','B1','B2']), 'HazardClassification'] = 'High'
result.loc[result['status'].isin(['SE','C']), 'HazardClassification'] = 'Moderate'
result.loc[result['status'].isin(['NH','E']), 'HazardClassification'] = 'Low'




result['CauseOfClassification'] = 'INI,D'
result.loc[result['status'].isin(['A','CH']), 'CauseOfClassification'] = 'CH,A'
result.loc[result['status'].isin(['LH','B1','B2']), 'CauseOfClassification'] = 'LH,B1,B2'
result.loc[result['status'].isin(['SE','C']), 'CauseOfClassification'] = 'SE,C'
result.loc[result['status'].isin(['NH','E']), 'CauseOfClassification'] = 'NH,E'

result['SourceName'] = 'US Environmental Protection Agency'
result['SourceURL'] = 'https://www.epa.gov/fera/dose-response-assessment-assessing-health-risks-associated-exposure-hazardous-air-pollutants'

result = result.drop(['status'], axis=1)

result.head(2)

,CAS Number,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,75-07-0,US,Environment Protection Aganecy (EPA),Health,Carcinegenity,Likely to be carcinogenic,High,"LH,B1,B2",US Environmental Protection Agency,https://www.epa.gov/fera/dose-response-assessm...
1,60-35-5,US,Environment Protection Aganecy (EPA),Health,Carcinegenity,Inadequate information or not evaluated,Very Low,"INI,D",US Environmental Protection Agency,https://www.epa.gov/fera/dose-response-assessm...
2,75-05-8,US,Environment Protection Aganecy (EPA),Health,Carcinegenity,Inadequate information or not evaluated,Very Low,"INI,D",US Environmental Protection Agency,https://www.epa.gov/fera/dose-response-assessm...
3,98-86-2,US,Environment Protection Aganecy (EPA),Health,Carcinegenity,Inadequate information or not evaluated,Very Low,"INI,D",US Environmental Protection Agency,https://www.epa.gov/fera/dose-response-assessm...
4,107-02-8,US,Environment Protection Aganecy (EPA),Health,Carcinegenity,Inadequate information or not evaluated,Very Low,"INI,D",US Environmental Protection Agency,https://www.epa.gov/fera/dose-response-assessm...
...,...,...,...,...,...,...,...,...,...,...
227,14807-96-6,US,Environment Protection Aganecy (EPA),Health,Carcinegenity,Inadequate information or not evaluated,Very Low,"INI,D",US Environmental Protection Agency,https://www.epa.gov/fera/dose-response-assessm...
228,7631-86-91,US,Environment Protection Aganecy (EPA),Health,Carcinegenity,Inadequate information or not evaluated,Very Low,"INI,D",US Environmental Protection Agency,https://www.epa.gov/fera/dose-response-assessm...
229,9002-84-0,US,Environment Protection Aganecy (EPA),Health,Carcinegenity,Inadequate information or not evaluated,Very Low,"INI,D",US Environmental Protection Agency,https://www.epa.gov/fera/dose-response-assessm...
230,65-85-0,US,Environment Protection Aganecy (EPA),Health,Carcinegenity,Inadequate information or not evaluated,Very Low,"INI,D",US Environmental Protection Agency,https://www.epa.gov/fera/dose-response-assessm...


In [ ]:
result.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [5]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:00:31.827578
